## 4. LangChain 入門（前半）

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

### 4.3. Models（LLMs・Chat Models）

In [2]:
import langchain
from langchain.llms import OpenAI

llm = OpenAI(model_name="text-davinci-003", temperature=0)

result = llm.predict("自己紹介してください。")
print(result)



はじめまして、私は○○と申します。現在、○○大学で学んでいます。趣味は料理や旅行などです。特に料理は好きで、毎日新しいレシピを試しています。今までに海外旅行も何回かしていて、異文化に触れるのが大好きです。今後も新しいことに挑戦していきたいと思っています。よろしくお願いします。


In [3]:
import langchain
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

result = chat.predict("自己紹介してください。")
print(result)

こんにちは、私はAIです。私はOpenAIが開発した自然言語処理モデルです。私の目的は、ユーザーが質問や要求をすると、最善の回答や応答を提供することです。私は様々なトピックについての情報を持っており、文法やスタイルの修正も行うことができます。どのようにお手伝いできますか？


### 4.4. Prompts（Prompt Templates）

In [4]:
from langchain.prompts import PromptTemplate

template = """
次のコマンドの概要を説明してください。

コマンド: {command}
"""

prompt = PromptTemplate(
    input_variables=["command"],
    template=template,
)

result = prompt.format(command="ls")
print(result)


次のコマンドの概要を説明してください。

コマンド: ls



### 4.5. Chains

In [5]:
import langchain
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

langchain.verbose = True

# Model を用意
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Prompt を用意
template = """
次のコマンドの概要を説明してください。

コマンド: {command}
"""
prompt = PromptTemplate(
    input_variables=["command"],
    template=template,
)

# Chain を作成
chain = LLMChain(llm=chat, prompt=prompt)

# 実行
result = chain.run("ls")
print(result)



> Entering new LLMChain chain...
Prompt after formatting:

次のコマンドの概要を説明してください。

コマンド: ls


> Finished chain.
lsコマンドは、UnixやUnix系のオペレーティングシステムで使用されるコマンドで、指定されたディレクトリ内のファイルやディレクトリの一覧を表示するために使用されます。デフォルトでは、現在のディレクトリの内容が表示されますが、オプションを使用して他のディレクトリの内容を表示することもできます。ファイルやディレクトリの名前、パーミッション、所有者、サイズなどの情報が表示されます。


In [6]:
import langchain
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

langchain.verbose = True

# Model を用意
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# 1 つ目の Prompt と Chain を用意
cot_template = """
以下の質問に回答してください。

### 質問 ###
{question}
### 質問終了 ###

ステップバイステップで考えましょう。
"""
cot_prompt = PromptTemplate(
    input_variables=["question"],
    template=cot_template,
)
cot_chain = LLMChain(llm=chat, prompt=cot_prompt)

# 2 つ目の Prompt と Chain を用意
summarize_template = """
入力を結論だけ一言に要約してください。

### 入力 ###
{input}
### 入力終了 ###
"""
summarize_prompt = PromptTemplate(
    input_variables=["input"],
    template=summarize_template,
)
summarize_chain = LLMChain(llm=chat, prompt=summarize_prompt)

# 2 つの Chain を直列に繋ぐ
cot_summarize_chain = SimpleSequentialChain(
    chains=[cot_chain, summarize_chain])

# 実行
result = cot_summarize_chain(
    "私は市場に行って10個のリンゴを買いました。隣人に2つ、修理工に2つ渡しました。それから5つのリンゴを買って1つ食べました。残りは何個ですか？")
print(result["output"])



> Entering new SimpleSequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

以下の質問に回答してください。

### 質問 ###
私は市場に行って10個のリンゴを買いました。隣人に2つ、修理工に2つ渡しました。それから5つのリンゴを買って1つ食べました。残りは何個ですか？
### 質問終了 ###

ステップバイステップで考えましょう。


> Finished chain.
1. 最初に市場で10個のリンゴを買いました。
2. 隣人に2つのリンゴを渡しました。残りは8個です。
3. 修理工に2つのリンゴを渡しました。残りは6個です。
4. さらに5つのリンゴを買いました。残りは11個です。
5. 1つのリンゴを食べました。残りは10個です。

したがって、残りのリンゴは10個です。


> Entering new LLMChain chain...
Prompt after formatting:

入力を結論だけ一言に要約してください。

### 入力 ###
1. 最初に市場で10個のリンゴを買いました。
2. 隣人に2つのリンゴを渡しました。残りは8個です。
3. 修理工に2つのリンゴを渡しました。残りは6個です。
4. さらに5つのリンゴを買いました。残りは11個です。
5. 1つのリンゴを食べました。残りは10個です。

したがって、残りのリンゴは10個です。
### 入力終了 ###


> Finished chain.
最初に10個のリンゴを買い、渡したり食べたりした後も残りは10個です。

> Finished chain.
最初に10個のリンゴを買い、渡したり食べたりした後も残りは10個です。


### 4.7. Prompts のより高度な機能（Output Parses）

In [7]:
import langchain
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field, validator
from typing import List

langchain.verbose = True

# Model を用意
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Prompt を用意
template = """料理のレシピを教えてください。

{format_instructions}

料理名: {dish}
"""

class Recipe(BaseModel):
    ingredients: List[str] = Field(description="材料")
    steps: List[str] = Field(description="手順")
parser = PydanticOutputParser(pydantic_object=Recipe)

prompt = PromptTemplate(
    template=template,
    input_variables=["dish"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

# Chain を作成
chain = LLMChain(llm=chat, prompt=prompt)

# 実行
output = chain.run(dish="カレー")
print("=== output ===")
print(output)



> Entering new LLMChain chain...
Prompt after formatting:
料理のレシピを教えてください。

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"title": "Ingredients", "description": "\u6750\u6599", "type": "array", "items": {"type": "string"}}, "steps": {"title": "Steps", "description": "\u624b\u9806", "type": "array", "items": {"type": "string"}}}, "required": ["ingredients", "steps"]}
```

料理名: カレー


> Finished chain.
=== output ===
{
  "ingredients": [
    "玉ねぎ",
    "にんじん",
    "じゃがいも",
    "牛肉",
    "カレールー",
    "水"
  ],
  "steps": [
    "玉ねぎ、にんじん、じゃがいもを切ります。",
    "牛肉を炒め

In [8]:
recipe = parser.parse(output)
print("=== recipe object ===")
print(recipe)

=== recipe object ===
ingredients=['玉ねぎ', 'にんじん', 'じゃがいも', '牛肉', 'カレールー', '水'] steps=['玉ねぎ、にんじん、じゃがいもを切ります。', '牛肉を炒めます。', '野菜を加えて炒めます。', '水を加えて煮込みます。', 'カレールーを加えて溶かします。', '煮込んで完成です。']
